# Experiments Results

In [30]:
!python -m pip install numpy pandas

In [31]:
import os 
import csv
import numpy as np
import pandas as pd
from functools import reduce
import datetime
pd.set_option('max_columns', 50)
pd.set_option("max_colwidth", 100)

In [32]:
JUEGO_NAME = "juego2"
JUEGO_DIR = "../experiments/cidetic/{}".format(JUEGO_NAME)

In [33]:
def traverse_paths(paths, name="results.csv", extension=".csv"):
    csvs = []
    for path in paths:
        for root, dirs_list, files_list in os.walk(path):
            for file_name in files_list:
                if file_name==name and os.path.splitext(file_name)[-1] == extension:
                    file_name_path = os.path.join(root, file_name)
                    csvs.append(pd.read_csv(file_name_path))
    return csvs

In [34]:
def sizeof_human_readable(num, suffix='B'):
    # if nan
    if num != num: return num 
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

def seconds_human_readable(seconds):
    if seconds != seconds: return seconds 
    return str(datetime.timedelta(seconds=seconds))

TIME_DURATION_UNITS = (
    ('week', 60*60*24*7),
    ('day', 60*60*24),
    ('hour', 60*60),
    ('min', 60),
    ('sec', 1)
)

def seconds_human_readable2(seconds):
    # if nan
    if seconds != seconds: return seconds 
    if seconds == 0: return 'inf'
    parts = []
    for unit, div in TIME_DURATION_UNITS:
        amount, seconds = divmod(int(seconds), div)
        if amount > 0:
            parts.append('{} {}{}'.format(amount, unit, "" if amount == 1 else "s"))
    return ', '.join(parts)

In [35]:
paths = [
    JUEGO_DIR,
]
experiments = pd.concat(traverse_paths(paths))
print("Experiments: ", len(experiments))
print("Streams: ", np.unique(experiments.stream))
experiments["pretrain_size_prop"] = experiments['pretrain_size_prop'].round(2).astype('category')
experiments.head()

Experiments:  27
Streams:  ['20ng' 'enron' 'mediamill']


,Unnamed: 0,model,model_id,stream,copies,stream_name,instances,model_data,ensemble,pretrain_size_prop,pretrain_size,window_size,train_size,batch_size,train_logs_max,start_time,success,error,end_time,time_seconds,model_size_kb,accuracy_per_label,hamming_loss,hamming_score,exact_match,accuracy_subset,log_loss,jaccard_index_eb_acc,example_based_accuracy,example_based_precision,example_based_recall,example_based_f1,micro_precision,micro_recall,micro_fscore,macro_precision,macro_recall,macro_fscore
0,0,Label Combination Hoeffding Tree,lcht,20ng,1,20ng,19300,"LabelCombinationHoeffdingTreeClassifier(binary_split=False, grace_period=200,\n ...",False,0,0,20,19300,965,100000,1.614692e+09,True,False,1.614700e+09,8550.891702,23360.791992,"[0.9509844559585492, 0.9470466321243524, 0.9569948186528497, 0.9493782383419689, 0.9494818652849...",0.066212,0.933788,0.315337,0.315337,22.573453,0.321290,0.933788,0.327254,0.321342,0.324271,0.344583,0.318276,0.330907,0.545952,0.318130,0.402008
0,0,OzaBagging (cc) / ecc - nb,oza_ml_cc_nb,mediamill,1,mediamill,43907,"ClassifierChain(base_estimator=NaiveBayes(nominal_attributes=None),\n n_targets=1...",OzaBaggingMLClassifier(base_estimator=ClassifierChain(base_estimator=NaiveBayes(nominal_attribut...,0,0,20,43907,2196,100000,1.614657e+09,True,False,1.614748e+09,91381.656371,273624.670898,"[0.8299587765048854, 0.9971986243651354, 0.6453868403671397, 0.9736032978796092, 0.9930307240303...",0.076928,0.923072,0.001184,0.001184,65.868848,0.246156,0.923072,0.291332,0.582843,0.388483,0.299384,0.578811,0.394643,0.048588,0.111934,0.067762
0,0,Classifier Chain - Naive Bayes,cc_nb,enron,1,enron,1702,"ClassifierChain(base_estimator=NaiveBayes(nominal_attributes=None),\n n_targets=5...",False,0,0,20,1702,86,100000,1.614692e+09,True,False,1.614693e+09,1625.572559,87093.701172,"[0.9847238542890717, 0.9623971797884842, 0.9964747356051704, 0.9847238542890717, 0.8037602820211...",0.080593,0.919407,0.017039,0.017039,53.297256,0.216015,0.919407,0.336517,0.352843,0.344486,0.363799,0.353043,0.358341,0.113169,0.097087,0.104513
0,0,Binary Relevance - Naive Bayes,br_nb,mediamill,1,mediamill,43907,"MultiOutputLearner(base_estimator=NaiveBayes(nominal_attributes=None),\n n_tar...",False,0,0,20,43907,2196,100000,1.614692e+09,True,False,1.614699e+09,7722.170464,19372.308594,"[0.8257908761700867, 0.7281754617714715, 0.595690892112875, 0.801330084041269, 0.636322226524244...",0.289343,0.710657,0.001298,0.001298,57.054128,0.091000,0.710657,0.093205,0.650681,0.163054,0.095790,0.672883,0.167705,0.061622,0.552959,0.110887
0,0,OzaBagging (br) / ebr - nb,oza_ml_br_nb,20ng,1,20ng,19300,"MultiOutputLearner(base_estimator=NaiveBayes(nominal_attributes=None),\n n_tar...",OzaBaggingMLClassifier(base_estimator=MultiOutputLearner(base_estimator=NaiveBayes(nominal_attri...,0,0,20,19300,965,100000,1.614657e+09,True,False,1.614708e+09,51296.346057,323657.671875,"[0.9506735751295337, 0.9421761658031088, 0.9504145077720207, 0.9621761658031088, 0.9468393782383...",0.046821,0.953179,0.152642,0.152642,4.111245,0.182024,0.953179,0.184554,0.213031,0.197773,0.632580,0.214332,0.320181,0.687076,0.213920,0.326259


In [36]:
starts_with = lambda prefix, c: any(c.startswith(sw) for sw in prefix) 
metrics_starting_with = lambda prefix: list(filter(lambda c: starts_with(prefix, c), experiments.columns.values.tolist()))
def primer_experimento(df):
    if len(df) > 1:
        print("Warning: se halló más de un experimento para este modelo y dataset\nSe toma solo el primero.")
    return df.iloc[0]

In [37]:
example_based_metrics = ["exact_match",  "hamming_score", "hamming_loss"]
example_based_metrics_prefix = ["example"]
example_based_metrics.extend(metrics_starting_with(example_based_metrics_prefix))
label_based_metrics_macro = []
label_based_metrics_macro_prefix = ["macro"]
label_based_metrics_macro.extend(metrics_starting_with(label_based_metrics_macro_prefix))
label_based_metrics_micro = []
label_based_metrics_micro_prefix = ["micro"]
label_based_metrics_micro.extend(metrics_starting_with(label_based_metrics_micro_prefix))
efficiency_metrics = ["model_size_kb", "time_seconds"]
other_metrics = ["log_loss"]

print(
    example_based_metrics,
    label_based_metrics_micro,
    label_based_metrics_macro,
    efficiency_metrics,
    other_metrics,
)

['exact_match', 'hamming_score', 'hamming_loss', 'example_based_accuracy', 'example_based_precision', 'example_based_recall', 'example_based_f1'] ['micro_precision', 'micro_recall', 'micro_fscore'] ['macro_precision', 'macro_recall', 'macro_fscore'] ['model_size_kb', 'time_seconds'] ['log_loss']


In [38]:
streams = ["enron", "20ng", "mediamill"]
df_group = experiments.loc[experiments["stream"].isin(streams)].groupby(["model_id", "stream"])
def apply_metrics(columns, df_group=df_group, from_col_id=0, to_col_id=None):
    return df_group[columns].apply(primer_experimento).unstack()

## Example-based Metrics

In [21]:
apply_metrics(example_based_metrics[:3])

0            exact_match                     hamming_score            \
stream              20ng     enron mediamill          20ng     enron   
model_id                                                               
br_nb           0.227979  0.017626  0.001298      0.952153  0.912423   
cc_nb           0.244249  0.017039  0.004646      0.954241  0.919407   
dwmc_br         0.171295  0.020564  0.001344      0.955697  0.938408   
dwmc_cc         0.166788  0.023502  0.013597      0.955570  0.939028   
lcht            0.315337  0.054642  0.048011      0.933788  0.928475   
me2_lcht        0.219948  0.039365  0.004054      0.954925  0.936135   
me_lcht         0.241140  0.032315  0.012390      0.953987  0.927677   
oza_ml_br_nb    0.152642  0.012338  0.001321      0.953179  0.928453   
oza_ml_cc_nb    0.114767  0.011751  0.001184      0.952008  0.938197   

0                      hamming_loss                      
stream       mediamill         20ng     enron mediamill  
model_id                                                 
br_nb         0.710657     0.047847  0.087577  0.289343  
cc_nb         0.904773     0.045759  0.080593  0.095227  
dwmc_br       0.800074     0.044303  0.061592  0.199926  
dwmc_cc       0.935624     0.044430  0.060972  0.064376  
lcht          0.957226     0.066212  0.071525  0.042774  
me2_lcht      0.928305     0.045075  0.063865  0.071695  
me_lcht       0.946128     0.046013  0.072323  0.053872  
oza_ml_br_nb  0.725642     0.046821  0.071547  0.274358  
oza_ml_cc_nb  0.923072     0.047992  0.061803  0.076928

In [23]:
apply_metrics(example_based_metrics[3:])

0            example_based_accuracy                      \
stream                         20ng     enron mediamill   
model_id                                                  
br_nb                      0.952153  0.912423  0.710657   
cc_nb                      0.954241  0.919407  0.904773   
dwmc_br                    0.955697  0.938408  0.800074   
dwmc_cc                    0.955570  0.939028  0.935624   
lcht                       0.933788  0.928475  0.957226   
me2_lcht                   0.954925  0.936135  0.928305   
me_lcht                    0.953987  0.927677  0.946128   
oza_ml_br_nb               0.953179  0.928453  0.725642   
oza_ml_cc_nb               0.952008  0.938197  0.923072   

0            example_based_precision                     example_based_recall  \
stream                          20ng     enron mediamill                 20ng   
model_id                                                                        
br_nb                       0.295144  0.326133  0.093205             0.372850   
cc_nb                       0.294158  0.336517  0.228897             0.339275   
dwmc_br                     0.187199  0.304725  0.129398             0.196813   
dwmc_cc                     0.178472  0.308451  0.350028             0.183342   
lcht                        0.327254  0.268508  0.488276             0.321342   
me2_lcht                    0.251930  0.388722  0.303500             0.276891   
me_lcht                     0.301599  0.379963  0.404023             0.360544   
oza_ml_br_nb                0.184554  0.320416  0.098524             0.213031   
oza_ml_cc_nb                0.135950  0.374990  0.291332             0.148705   

0                                example_based_f1                      
stream           enron mediamill             20ng     enron mediamill  
model_id                                                               
br_nb         0.365481  0.650681         0.329477  0.344688  0.163054  
cc_nb         0.352843  0.485245         0.315110  0.344486  0.311062  
dwmc_br       0.183242  0.622667         0.191886  0.228861  0.214269  
dwmc_cc       0.185023  0.466489         0.180874  0.231301  0.399953  
lcht          0.126535  0.377923         0.324271  0.172010  0.426070  
me2_lcht      0.302834  0.515855         0.263822  0.340444  0.382159  
me_lcht       0.350953  0.417533         0.328448  0.364882  0.410667  
oza_ml_br_nb  0.268605  0.649196         0.197773  0.292232  0.171084  
oza_ml_cc_nb  0.227256  0.582843         0.142042  0.283003  0.388483

## Label-based Metrics

In [39]:
apply_metrics(label_based_metrics_macro)

0            macro_precision                     macro_recall            \
stream                  20ng     enron mediamill         20ng     enron   
model_id                                                                  
br_nb               0.604366  0.106494  0.061622     0.372708  0.110179   
cc_nb               0.667077  0.113169  0.065455     0.339543  0.097087   
dwmc_br             0.781260  0.120868  0.063786     0.196317  0.031024   
dwmc_cc             0.814060  0.113886  0.068133     0.183000  0.029983   
lcht                0.545952  0.005336  0.073886     0.318130  0.016178   
me2_lcht            0.698825  0.142894  0.096075     0.275952  0.058921   
me_lcht             0.648793  0.114156  0.073763     0.360567  0.082266   
oza_ml_br_nb        0.687076  0.099991  0.062030     0.213920  0.067803   
oza_ml_cc_nb        0.753274  0.118776  0.048588     0.149538  0.046321   

0                      macro_fscore                      
stream       mediamill         20ng     enron mediamill  
model_id                                                 
br_nb         0.552959     0.461075  0.108305  0.110887  
cc_nb         0.149993     0.450024  0.104513  0.091138  
dwmc_br       0.418315     0.313785  0.049374  0.110693  
dwmc_cc       0.110687     0.298825  0.047469  0.084346  
lcht          0.029855     0.402008  0.008025  0.042527  
me2_lcht      0.173468     0.395664  0.083438  0.123661  
me_lcht       0.071878     0.463529  0.095622  0.072808  
oza_ml_br_nb  0.530654     0.326259  0.080810  0.111076  
oza_ml_cc_nb  0.111934     0.249538  0.066649  0.067762

In [40]:
apply_metrics(label_based_metrics_micro)

0            micro_precision                     micro_recall            \
stream                  20ng     enron mediamill         20ng     enron   
model_id                                                                  
br_nb               0.551691  0.330442  0.095790     0.373017  0.364348   
cc_nb               0.597062  0.363799  0.223674     0.339830  0.353043   
dwmc_br             0.772763  0.552489  0.131111     0.196606  0.177565   
dwmc_cc             0.795976  0.569061  0.327806     0.183311  0.179130   
lcht                0.344583  0.282797  0.509415     0.318276  0.079478   
me2_lcht            0.644350  0.498374  0.304682     0.276275  0.293217   
me_lcht             0.585649  0.417202  0.383906     0.360880  0.339130   
oza_ml_br_nb        0.632580  0.411558  0.100509     0.214332  0.284870   
oza_ml_cc_nb        0.644156  0.533538  0.299384     0.149872  0.242087   

0                      micro_fscore                      
stream       mediamill         20ng     enron mediamill  
model_id                                                 
br_nb         0.672883     0.445092  0.346567  0.167705  
cc_nb         0.484905     0.433133  0.358341  0.306135  
dwmc_br       0.642397     0.313461  0.268755  0.217775  
dwmc_cc       0.462570     0.297994  0.272487  0.383699  
lcht          0.342472     0.330907  0.124084  0.409586  
me2_lcht      0.510801     0.386733  0.369211  0.381692  
me_lcht       0.402612     0.446577  0.374137  0.393037  
oza_ml_br_nb  0.670859     0.320181  0.336691  0.174825  
oza_ml_cc_nb  0.578811     0.243167  0.333054  0.394643

## Efficiency Metrics

In [41]:
def to_human(x):
    column, _ = x.name
    if column=='model_size_kb':
        return x.apply(lambda size: sizeof_human_readable(size))
    if column=='time_seconds':
        return x.apply(lambda seconds: seconds_human_readable(seconds))

apply_metrics(efficiency_metrics).apply(lambda x: to_human(x))

0            model_size_kb                         time_seconds  \
stream                20ng     enron mediamill             20ng   
model_id                                                          
br_nb              31.6KiB   82.9KiB   18.9KiB   1:43:26.207637   
cc_nb              31.9KiB   85.1KiB   26.7KiB   1:45:00.857116   
dwmc_br           305.5KiB  782.9KiB  185.3KiB  21:23:37.063584   
dwmc_cc           308.4KiB  802.7KiB  262.4KiB  21:03:02.934515   
lcht               22.8KiB  284.7KiB  306.0KiB   2:22:30.891702   
me2_lcht           84.7KiB  359.2KiB  262.2KiB   5:36:55.718539   
me_lcht            86.4KiB  452.8KiB  351.7KiB   5:41:58.628873   
oza_ml_br_nb      316.1KiB  826.8KiB  189.2KiB  14:14:56.346057   
oza_ml_cc_nb      319.0KiB  847.6KiB  267.2KiB  14:35:33.062475   

0                                                     
stream                 enron               mediamill  
model_id                                              
br_nb         0:21:28.042753          2:08:42.170464  
cc_nb         0:27:05.572559          2:52:36.012434  
dwmc_br       5:15:03.250381   1 day, 4:21:27.713977  
dwmc_cc       5:05:56.635203  1 day, 12:49:14.373212  
lcht          0:58:16.999835         19:05:49.031182  
me2_lcht      2:03:59.325276   1 day, 6:27:51.215345  
me_lcht       1:45:51.230247   1 day, 0:47:17.156675  
oza_ml_br_nb  3:41:00.620831         20:13:01.505864  
oza_ml_cc_nb  4:03:16.037589   1 day, 1:23:01.656371

In [42]:
apply_metrics(other_metrics)

0              log_loss                       
stream             20ng       enron  mediamill
model_id                                      
br_nb          4.545242   52.217780  57.054128
cc_nb          5.520631   53.297256  80.425498
dwmc_br        3.636037   49.166403  60.076141
dwmc_cc        3.692041   49.086484  81.425633
lcht          22.573453  105.291811  97.272378
me2_lcht       5.199270   57.350604  75.505601
me_lcht        5.274037   57.932464  89.570941
oza_ml_br_nb   4.111245   53.802497  57.116614
oza_ml_cc_nb   4.287778   59.963079  65.868848